## 데이터마이닝 프로젝트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# 데이터 로드

FP_PATH    = "floating_population.csv"   # 유동인구
SALES_PATH = "sales.csv"                 # 매출
STORE_PATH = "store.csv"                 # 점포


In [ ]:
def read_korean_csv(path: str) -> pd.DataFrame:
    """Try common Korean encodings."""
    for enc in ("utf-8", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path)

fp    = read_korean_csv(FP_PATH)
fp = fp[fp["기준_년분기_코드"] // 10 == 2024]
sales = read_korean_csv(SALES_PATH)
store = read_korean_csv(STORE_PATH)

f = lambda x: display(x["기준_년분기_코드"].unique())
f(fp)
f(sales)
f(store)


In [ ]:
# 열 확인

def get_columns(df):
    display(df.columns)
get_columns(fp)
get_columns(sales)
get_columns(store)

In [ ]:
#데이터 전처리
def make_numeric(df, drop_cols):
    return [c for c in df.columns if c not in drop_cols and df[c].dtype != 'object']

sales_num = make_numeric(sales, ["행정동_코드", "기준_년분기_코드"])
store_num = make_numeric(store, ["행정동_코드", "기준_년분기_코드"])

sales_agg = (sales
             .groupby(["기준_년분기_코드", "행정동_코드"])[sales_num]
             .sum()
             .reset_index())

store_agg = (store
             .groupby(["기준_년분기_코드", "행정동_코드"])[store_num]
             .median() # 0 값이 많아 median과 차이가 있는데 어떤 걸로 할지?
             .reset_index())

# merged 건들면 안됨!
merged = (fp
          .merge(sales_agg, on=["기준_년분기_코드", "행정동_코드"], how="left")
          .merge(store_agg, on=["기준_년분기_코드", "행정동_코드"], how="left"))
merged = merged.dropna(axis=0)
merged.isna().sum().sum()


In [ ]:
seoul_gu_code_map = {
    '11110': '종로구',
    '11140': '중구',
    '11170': '용산구',
    '11200': '성동구',
    '11215': '광진구',
    '11230': '동대문구',
    '11260': '중랑구',
    '11290': '성북구',
    '11305': '강북구',
    '11320': '도봉구',
    '11350': '노원구',
    '11380': '은평구',
    '11410': '서대문구',
    '11440': '마포구',
    '11470': '양천구',
    '11500': '강서구',
    '11530': '구로구',
    '11545': '금천구',
    '11560': '영등포구',
    '11590': '동작구',
    '11620': '관악구',
    '11650': '서초구',
    '11680': '강남구',
    '11710': '송파구',
    '11740': '강동구'
}

import pandas as pd

# 예시 DataFrame
# merged = pd.read_csv("your_file.csv")  # 실제 사용할 때

# 행정동_코드를 문자열로 변환
merged['행정동_코드'] = merged['행정동_코드'].astype(str)

# 시군구 코드 추출 (앞 5자리)
merged['시군구코드'] = merged['행정동_코드'].str[:5]

# 자치구명 매핑
merged['자치구'] = merged['시군구코드'].map(seoul_gu_code_map)

# 결과 확인
print(merged[['행정동_코드', '시군구코드', '자치구']])

In [ ]:
selected_df = pd.DataFrame()
selected_df["총_유동인구_수"] = merged["총_유동인구_수"]
selected_df["청년층_유동인구_비율"] = (merged["연령대_20_유동인구_수"] + merged["연령대_30_유동인구_수"])/merged["총_유동인구_수"]
selected_df["청년_매출_비율"] = (merged["연령대_20_매출_건수"] + merged["연령대_30_매출_건수"]) / merged["당월_매출_건수"]
selected_df["주말_매출_비율"] =  merged["주중_매출_건수"]/ merged["주말_매출_건수"]
selected_df["행정동_코드"] = merged["행정동_코드"]
selected_df["야간유동인구비율"] = (merged["시간대_17_21_유동인구_수"] 
                            + merged["시간대_21_24_유동인구_수"] 
                            + merged["시간대_00_06_유동인구_수"]) / merged["총_유동인구_수"]
selected_df["점포당_월매출액"] = merged["당월_매출_금액"] / merged["유사_업종_점포_수"]
selected_df["객단가"] = merged["당월_매출_금액"] / merged["당월_매출_건수"]
selected_df["개업_율"] = merged["개업_율"]
selected_df["프랜차이즈_점포_비율"] = merged["프랜차이즈_점포_수"] / merged["유사_업종_점포_수"]


# 4.1 이상치 처리
def remove_outliers(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# 이상치가 많이 발생할 수 있는 변수들 선택
outlier_columns = ['총_유동인구_수', '청년층_유동인구_비율', '청년_매출_비율', '주말_매출_비율', '야간유동인구비율', '점포당_월매출액', '객단가', '개업_율', '프랜차이즈_점포_비율']
selected_df = remove_outliers(selected_df, outlier_columns)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(selected_df)

pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

pca3 = PCA(n_components=3)
X_pca3 = pca3.fit_transform(X_scaled)


In [ ]:
# 클러스터 로직
kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
selected_df['cluster'] = kmeans.fit_predict(X_pca2)

In [ ]:
# 2d 시각화
plt.scatter(X_pca2[:, 0], X_pca2[:, 1], c=selected_df['cluster'], cmap='viridis')
plt.colorbar()
plt.show()


In [ ]:
# 클러스터별 통계 분석
cluster_stats = selected_df.groupby('cluster').agg({
    "총_유동인구_수": 'mean',
    "청년층_유동인구_비율": 'mean',
    "청년_매출_비율": 'mean',
    "주말_매출_비율": 'mean',
    "야간유동인구비율": 'mean',
    "점포당_월매출액": 'mean',
    "객단가": 'mean',
    "개업_율": 'mean',
    "프랜차이즈_점포_비율": 'mean',
}).round(2)

# 각 클러스터별 샘플 데이터 확인 (각 클러스터에서 3개씩)
cluster_samples = {}
for cluster in selected_df['cluster'].unique():
    cluster_samples[cluster] = selected_df[selected_df['cluster'] == cluster].sample(10)

# 결과 출력
print("=== 클러스터별 평균 통계 ===")
display(cluster_stats)

print("\n=== 클러스터별 샘플 데이터 (원래 스케일) ===")
for cluster, samples in cluster_samples.items():
    print(f"\n클러스터 {cluster}의 샘플:")
    display(samples)

# 각 클러스터별 구 분포 분석
print("\n=== 클러스터별 구 분포 (상위 3개) ===")
for cluster in selected_df['cluster'].unique():
    cluster_data = selected_df[selected_df['cluster'] == cluster]
    # 행정동 코드의 앞 5자리가 구 코드
    gu_counts = cluster_data['행정동_코드'].astype(str).str[:5].value_counts()
    
    print(f"\n클러스터 {cluster}의 주요 구:")
    for gu_code, count in gu_counts.head(3).items():
        gu_name = seoul_gu_code_map.get(gu_code, '알 수 없음')
        print(f"- {gu_name}: {count}개 행정동")

#  실루엣 계수
from sklearn.metrics import silhouette_score

silhouette_scores = []

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_pca3)
    labels = kmeans.labels_
    score = silhouette_score(X_pca3, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Nanum Gothic'
plt.rcParams['axes.unicode_minus'] = False
import pandas as pd

# 각 클러스터별 구 분포 분석 및 필터링
cluster_gu_counts = {}
for cluster in selected_df['cluster'].unique():
    cluster_data = selected_df[selected_df['cluster'] == cluster]
    gu_counts = cluster_data['행정동_코드'].astype(str).str[:5].value_counts()
    
    # 최소 5개 이상인 구만 선택
    significant_gu = gu_counts[gu_counts >= 5]
    cluster_gu_counts[cluster] = significant_gu

# 각 구별 클러스터 분포 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 구별 클러스터 분포 데이터 생성
gu_cluster_data = []
for cluster, gu_counts in cluster_gu_counts.items():
    for gu_code, count in gu_counts.items():
        gu_name = seoul_gu_code_map.get(gu_code, '알 수 없음')
        gu_cluster_data.append({
            '구': gu_name,
            '클러스터': cluster,
            '행정동 수': count
        })

gu_cluster_df = pd.DataFrame(gu_cluster_data)

# 시각화
plt.figure(figsize=(15, 8))
sns.barplot(data=gu_cluster_df, x='구', y='행정동 수', hue='클러스터')
plt.xticks(rotation=45, ha='right')
plt.title('구별 클러스터 분포')
plt.tight_layout()
plt.show()

# 각 클러스터별 주요 구 시각화
plt.figure(figsize=(15, 8))
for cluster in selected_df['cluster'].unique():
    cluster_data = gu_cluster_df[gu_cluster_df['클러스터'] == cluster]
    plt.bar(cluster_data['구'], cluster_data['행정동 수'], label=f'클러스터 {cluster}')

plt.xticks(rotation=45, ha='right')
plt.title('클러스터별 구 분포')
plt.legend()
plt.tight_layout()
plt.show()

# 히트맵으로 시각화
pivot_df = gu_cluster_df.pivot(index='구', columns='클러스터', values='행정동 수').fillna(0)
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_df, annot=True, cmap='YlOrRd', fmt='.0f')
plt.title('구별 클러스터 분포 히트맵')
plt.tight_layout()
plt.show()

In [ ]:
# 라벨링된 데이터 추가

import pandas as pd
from scipy.spatial import ConvexHull

# label.xlsx에서 라벨 정보 읽기
label_df = pd.read_excel('label.xlsx')

label_map = dict(zip(label_df['행정동_코드'], label_df['라벨']))

# selected_df에 라벨 컬럼 추가 (없으면 NaN)
selected_df['라벨'] = selected_df['행정동_코드'].map(label_map)

plt.figure(figsize=(15, 10))

unique_gu_codes = selected_df['행정동_코드'].astype(str).str[:5].unique()
gu_colors = {gu_code: plt.cm.tab20(i % 20) for i, gu_code in enumerate(unique_gu_codes)}

for cluster in selected_df['cluster'].unique():
    cluster_mask = selected_df['cluster'] == cluster
    cluster_points = X_pca2[cluster_mask]
    if len(cluster_points) > 2:
        hull = ConvexHull(cluster_points)
        plt.plot(cluster_points[hull.vertices, 0], cluster_points[hull.vertices, 1], 'k--', alpha=0.3)
        plt.fill(cluster_points[hull.vertices, 0], cluster_points[hull.vertices, 1], alpha=0.1)
    
    for gu_code in unique_gu_codes:
        gu_mask = selected_df['행정동_코드'].astype(str).str[:5] == gu_code
        combined_mask = cluster_mask & gu_mask
        # 라벨별로 마커 다르게
        star_mask = combined_mask & (selected_df['라벨'] == 1)
        x_mask = combined_mask & (selected_df['라벨'] == 0)
        none_mask = combined_mask & (selected_df['라벨'].isna())
        if star_mask.any():
            plt.scatter(
                X_pca2[star_mask, 0],
                X_pca2[star_mask, 1],
                c=[gu_colors[gu_code]],
                marker='*',
                s=600,                # 훨씬 크게!
                edgecolor='black',
                linewidths=2.5,       # 두꺼운 테두리
                alpha=1.0,
                zorder=10,
                label=None
            )
        if x_mask.any():
            plt.scatter(
                X_pca2[x_mask, 0],
                X_pca2[x_mask, 1],
                c=[gu_colors[gu_code]],
                marker='x',
                s=400,                # 훨씬 크게!
                edgecolor='black',
                linewidths=4,         # 두꺼운 테두리
                alpha=1.0,
                zorder=9,
                label=None
            )
        if none_mask.any():
            plt.scatter(
                X_pca2[none_mask, 0],
                X_pca2[none_mask, 1],
                c=[gu_colors[gu_code]],
                marker='o',
                s=80,
                alpha=0.5,
                zorder=1,
                label=None
            )

# 범례 추가 (구별 색상)
handles = [plt.Line2D([0], [0], marker='o', color='w', 
                     markerfacecolor=gu_colors[gu_code], 
                     markersize=10, 
                     label=seoul_gu_code_map.get(gu_code, "알 수 없음"))
          for gu_code in unique_gu_codes]

# 마커별 범례 추가
handles += [
    plt.Line2D([0], [0], marker='*', color='w', markerfacecolor='gray', markersize=18, label='라벨 1 (별)'),
    plt.Line2D([0], [0], marker='x', color='black', markerfacecolor='black', markersize=16, label='라벨 0 (X)'),
    plt.Line2D([0], [0], marker='o', color='gray', markerfacecolor='gray', markersize=10, label='라벨 없음')
]

plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('PCA 기반 클러스터 시각화 (구별 색상, 라벨별 마커)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.tight_layout()
plt.show()

In [ ]:
# 라벨된 데이터들 확인하기

selected_df['행정동_코드'] = selected_df['행정동_코드'].astype(str)
label_df['행정동_코드'] = label_df['행정동_코드'].astype(str)

# 다시 라벨 매핑
selected_df['라벨'] = selected_df['행정동_코드'].map(dict(zip(label_df['행정동_코드'], label_df['라벨'])))
print(selected_df['라벨'].value_counts(dropna=False))
labeled_df = selected_df[selected_df['라벨'].isin([0, 1])]
print(len(labeled_df))
print(labeled_df[['행정동_코드', '라벨']].head())

# 라벨이 있는 데이터만 추출
labeled_df = selected_df[selected_df['라벨'].isin([0, 1])]

# 각 라벨별로 클러스터 분포 출력
for label_value in [0.0, 1.0]:
    print(f"\n라벨 {label_value}인 데이터의 클러스터 분포:")
    label_cluster_counts = labeled_df[labeled_df['라벨'] == label_value]['cluster'].value_counts().sort_index()
    for cluster, count in label_cluster_counts.items():
        print(f"  클러스터 {cluster}: {count}개")

In [ ]:
# 클러스터별 1.0, 0.0 개수와 비율 출력
print("클러스터별 1.0/0.0 비율 (1.0 개수 / 0.0 개수):")
for cluster in sorted(labeled_df['cluster'].unique()):
    cluster_data = labeled_df[labeled_df['cluster'] == cluster]
    count_1 = (cluster_data['라벨'] == 1.0).sum()
    count_0 = (cluster_data['라벨'] == 0.0).sum()
    ratio = count_1 / count_0 if count_0 > 0 else float('inf')
    print(f"클러스터 {cluster}: 1.0 = {count_1}, 0.0 = {count_0}, 비율(1.0/0.0) = {ratio:.2f}")

In [ ]:
# 클러스터2 데이터 추출
print(selected_df[selected_df['cluster'] == 2]['행정동_코드'].value_counts())